In [66]:
import os

import numpy as np
import pandas as pd
from fuzzywuzzy import process, fuzz
import requests
from tqdm import tqdm
import plotly.express as px

tqdm.pandas()
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [4]:
def get_neptun_fdl(neptun_request):
    o_auth_ur_l = "https://host.sdakft.hu/mome_adatinterface/FDLSzuroFuttato?FDLNeve=" + neptun_request
    headers_get = {'Accept': 'application/json'}
    response_body = requests.get(url=o_auth_ur_l, headers=headers_get,
                                 auth=requests.auth.HTTPBasicAuth("ISBF3Z", "123456"))
    print(response_body)
    return pd.json_normalize(response_body.json()["Eredmeny"])

In [5]:
df_felvi = pd.read_excel('data/Jelentkezések_20230307.xlsx')

In [6]:
df_felvi = df_felvi[['Születési dátum', 'Felvételi azonosítószám', 'Név', 'Anyja neve']].drop_duplicates()

In [7]:
df_neptun = get_neptun_fdl('jrysztv_hallgatok')
df_neptun.columns = ['édesanyja neve', 'hallgató neve', 'neptunkód', 'születési dátum', 'születési hely',
                     'felvétel dátuma',
                     'felvétel féléve', 'felvétel módja', 'hallgató képzése', 'felvételi azonosító']

<Response [200]>


In [8]:
df_neptun = df_neptun[df_neptun['felvételi azonosító'].notna()]

In [9]:
df_neptun['felvételi azonosító'] = df_neptun['felvételi azonosító'].astype('Int64')

In [10]:
df_felvi_neptun = pd.merge(df_felvi.rename(
    {'Anyja neve': 'édesanyja neve', 'Név': 'hallgató neve', 'Felvételi azonosítószám': 'felvételi azonosító',
     'Születési dátum': 'születési dátum'}, axis=1),
    df_neptun, on='felvételi azonosító', how='outer', indicator=True)

In [11]:
df_felvi_neptun['felvett'] = df_felvi_neptun['_merge'] != 'left_only'  # aki neptunos, az fel van véve
df_felvi_neptun['jelentkezett'] = True  # Aki felvis, vagy felvett, az jelentkezett by definition :)

In [12]:
df_felvi_neptun.groupby('_merge').size()

_merge
left_only     1961
right_only       0
both           178
dtype: int64

In [13]:
df_felvi_neptun.drop('_merge', inplace=True, axis=1)

In [14]:
df_felvi_neptun['hallgató neve'] = df_felvi_neptun['hallgató neve_x'].fillna(
    df_felvi_neptun['hallgató neve_y']).str.strip()
df_felvi_neptun['édesanyja neve'] = df_felvi_neptun['édesanyja neve_x'].fillna(
    df_felvi_neptun['édesanyja neve_y']).str.strip()
df_felvi_neptun['születési dátum'] = df_felvi_neptun['születési dátum_x'].fillna(
    df_felvi_neptun['születési dátum_y']).str.strip()

In [15]:
df_felvi_neptun.drop(['hallgató neve_x', 'hallgató neve_y', 'édesanyja neve_x', 'édesanyja neve_y'], axis=1,
                     inplace=True)

In [16]:
df_felvi_neptun['id'] = df_felvi_neptun['hallgató neve'].str.cat(
    df_felvi_neptun[['édesanyja neve', 'születési dátum']].values, sep=', ')

# Premome
## Tisztítás

In [17]:
premome_data = pd.read_excel('data/premome_participants.xlsx', sheet_name=None)

In [18]:
df_premome = pd.concat(premome_data.values(), keys=premome_data.keys(), names=['tanfolyam']).reset_index().drop(
    'level_1', axis=1)

In [19]:
kurzusmapping = {'Textil szakirány': 'textiltervezés', 'Tervezőgrafika': 'tervezőgrafika',
                 'Tervezőgrafika - Haladó': 'tervezőgrafika',
                 'Tervezőgrafika - Kezdő': "tervezőgrafika", 'Textiltervezés': 'textiltervezés',
                 'Designkultúra': 'designkultúra',
                 'Média design': 'média design', 'Média Design': 'média design',
                 'Fotográfia': 'fotográfia', 'Fotográfia szakirány': 'fotográfia',
                 'Formatervezés': 'formatervezés', 'Formatervezés szakirány': 'formatervezés szakirány',
                 'Tárgyalkotás': 'tárgyalkotás', '7 alkalmas textil': 'textiltervezés',
                 'Rajzbérlet 19 alkalmas': 'rajz', 'Rajzbérlet 19 alkalmas ': 'rajz', 'Rajzbérlet 10 alkalmas': 'rajz',
                 'Rajzbérlet 10 alkalmas ': 'rajz', 'Rajz': 'rajz', '7 alkalmas rajz': 'rajz',
                 'Délelőtti rajz': 'rajz', 'Animáció': 'animáció', 'Építészet': 'építészet',
                 'Animáció szakirány': 'animáció', 'Tervezőgrafika szakirány': 'tervezőgrafika',
                 '7 alklamas rajz': 'rajz'}

In [20]:
pd.DataFrame(pd.Series(kurzusmapping)).reset_index().rename({'index': 'eredeti név', 0: 'harmonizált név'},
                                                            axis=1).sort_values('harmonizált név').to_excel(
    'results/harmonizálás.xlsx')

In [21]:
df_premome['kurzus neve (harmonizált)'] = df_premome['kurzus neve'].replace(kurzusmapping)

In [22]:
df_premome['kurzus azonosító'] = (df_premome['tanfolyam'] + ', ' + df_premome['kurzus neve']).str.lower()

In [23]:
df_premome['születési idő'] = df_premome['születési idő'].dt.strftime('%Y-%m-%d')
df_premome.rename({'születési idő': 'születési dátum'}, axis=1, inplace=True)

## Fuzzy merge előkészítés

In [24]:
df_premome['résztvevő neve'] = df_premome['résztvevő neve'].str.strip()
df_premome['édesanyja neve'] = df_premome['édesanyja neve'].str.strip()

In [25]:
df_premome['id'] = df_premome['résztvevő neve'].str.cat(df_premome[['édesanyja neve', 'születési dátum']].values,
                                                        sep=', ')

In [26]:
def get_closest_match(row, column1, df2, column2, scorer):
    closest_match_tuple = process.extractOne(row[column1], df2[column2], scorer=scorer)
    return pd.Series([closest_match_tuple[0], closest_match_tuple[1]])


def fuzzy_match(df1, df2, column1, column2, scorer=None, score_value=90):
    """
    Fuzzy match columns of two dataframes and return the closest match and score.

    Parameters:
    - df1, df2: Dataframes to match.
    - column1, column2: The columns in df1 and df2 respectively to match on.
    - scorer: A scoring function from fuzzywuzzy.fuzz. Default is fuzz.WRatio.

    Returns:
    - df1 with two new columns: 'closest_match' and 'score'.
    """
    if scorer is None:
        scorer = fuzz.WRatio

    df1[['closest_match', 'score']] = df1.progress_apply(
        lambda row: get_closest_match(row, column1, df2, column2, scorer),
        axis=1
    )
    df1.loc[df1['score'] < score_value, ['score', 'closest_match']] = (np.nan, None)
    return df1


df_premome = fuzzy_match(df_premome, df_felvi_neptun, 'id', 'id')

100%|██████████| 296/296 [02:48<00:00,  1.76it/s]


In [27]:
df_premome.rename({'closest_match': 'id neptun'}, axis=1, inplace=True)

In [28]:
df_premome.drop('score', axis=1, inplace=True)

In [29]:
df_premome['id'] = df_premome['id neptun'].fillna(df_premome['id'])

In [30]:
df = pd.merge(df_felvi_neptun, df_premome[['id', 'résztvevő neve', 'kurzus neve (harmonizált)', 'kurzus azonosító']],
              how='outer', indicator=True, on='id').rename({'kurzus neve (harmonizált)': 'kurzus neve'}, axis=1)

In [31]:
df['premomés'] = df['_merge'] != 'left_only'

In [32]:
df.groupby(['premomés', 'jelentkezett', 'felvett'], dropna=False).size()

premomés  jelentkezett  felvett
False     True          False      1884
                        True        150
True      True          False       113
                        True         52
          NaN           NaN         131
dtype: int64

In [33]:
df['jelentkezett'] = df['jelentkezett'].fillna(False)
df['felvett'] = df['felvett'].fillna(False)

In [34]:
df.drop(['_merge', 'felvétel dátuma', 'felvétel módja'], axis=1, inplace=True)

In [35]:
df.loc[df['felvett'], 'felvételi kategória'] = 'felvett'
df.loc[(~df['felvett']) & df['jelentkezett'], 'felvételi kategória'] = 'nem felvett'
df.fillna({'felvételi kategória': 'nem jelentkezett'}, inplace=True)

In [36]:
df['név'] = df['hallgató neve'].fillna(df['résztvevő neve'])

In [37]:
df.drop(['hallgató neve', 'résztvevő neve'], axis=1, inplace=True)

In [38]:
df['előkészítő típusa'] = df['premomés'].replace({True: 'premomés', False: 'nem premomés'})

In [39]:
df_totals = df.drop_duplicates(['id'])

In [49]:
pd.DataFrame(df_totals.groupby(['előkészítő típusa', 'felvételi kategória'], dropna=False).size()).reset_index().pivot(
    index='felvételi kategória', columns='előkészítő típusa', values=0).to_excel('results/felvételi arányok.xlsx')

In [50]:
pd.DataFrame(df_totals.groupby(['hallgató képzése', 'előkészítő típusa', 'felvételi kategória'],
                               dropna=False).size()).reset_index().pivot(
    index=['felvételi kategória', 'hallgató képzése'], columns='előkészítő típusa', values=0).to_excel(
    'results/felvételi arányok hallgató képzése szerint.xlsx')

In [135]:
def gen_application_admission_dfs(dataframe, coursevar_name='kurzus azonosító'):
    prepcourse_df = pd.DataFrame(
        dataframe.groupby([coursevar_name, 'előkészítő típusa', 'felvételi kategória'],
                          dropna=False).size()).reset_index().pivot(
        index=[coursevar_name, 'felvételi kategória'], columns='előkészítő típusa',
        values=0)  #.to_excel('results/felvételi arányok kurzus azonosító szerint.xlsx')
    prepcourse_df.reset_index(inplace=True)
    prepcourse_df.loc[prepcourse_df[coursevar_name].isna(), coursevar_name] = 'nem járt'
    prepcourse_df.set_index([coursevar_name, 'felvételi kategória'], inplace=True)
    prepcourse_df = prepcourse_df.join(
        prepcourse_df.fillna(0).groupby(coursevar_name, dropna=False).sum().sum(axis=1).rename(
            'teljes létszám')).fillna(0)
    prepcourse_df = prepcourse_df.reset_index().set_index('felvételi kategória').drop('nem jelentkezett',
                                                                                      axis=0).reset_index().set_index(
        [coursevar_name, 'felvételi kategória'])
    prepcourse_df = prepcourse_df.join(
        prepcourse_df[['premomés', 'nem premomés']].fillna(0).groupby(coursevar_name, dropna=False).sum().sum(
            axis=1).rename(
            'felvételiző létszám')).fillna(0)
    prepcourse_df.reset_index(inplace=True)
    prepcourse_df.loc[
        prepcourse_df['felvételi kategória']
        == 'felvett', 'felvételi ráta'] = (prepcourse_df.loc[
                                               prepcourse_df['felvételi kategória'] == 'felvett', 'nem premomés'] +
                                           prepcourse_df.loc[prepcourse_df[
                                                                 'felvételi kategória'] == 'felvett', 'premomés']) / \
                                          prepcourse_df.loc[prepcourse_df[
                                                                'felvételi kategória'] == 'felvett', 'felvételiző létszám']
    prepcourse_df.loc[
        prepcourse_df['felvételi kategória']
        == 'felvett', 'jelentkezési ráta'] = prepcourse_df.loc[prepcourse_df[
                                                                   'felvételi kategória'] == 'felvett', 'felvételiző létszám'] / \
                                             prepcourse_df.loc[prepcourse_df[
                                                                   'felvételi kategória'] == 'felvett', 'teljes létszám']
    prepcourse_df.set_index([coursevar_name, 'felvételi kategória'], inplace=True)
    prepcourse_df = prepcourse_df.fillna(0)
    return prepcourse_df

In [136]:
gen_application_admission_dfs(df)

nem premomés  \
kurzus azonosító                         felvételi kategória                 
nem járt                                 felvett                     150.0   
                                         nem felvett                1884.0   
7_alkalmas_őszi_2022, 7 alkalmas textil  felvett                       0.0   
                                         nem felvett                   0.0   
7_alkalmas_őszi_2022, 7 alklamas rajz    felvett                       0.0   
                                         nem felvett                   0.0   
előkészítő_2022, animáció                nem felvett                   0.0   
előkészítő_2022, designkultúra           felvett                       0.0   
előkészítő_2022, délelőtti rajz          nem felvett                   0.0   
előkészítő_2022, formatervezés           felvett                       0.0   
                                         nem felvett                   0.0   
előkészítő_2022, fotográfia              felvett                       0.0   
                                         nem felvett                   0.0   
előkészítő_2022, média design            felvett                       0.0   
                                         nem felvett                   0.0   
előkészítő_2022, rajzbérlet 10 alkalmas  felvett                       0.0   
                                         nem felvett                   0.0   
előkészítő_2022, rajzbérlet 19 alkalmas  felvett                       0.0   
                                         nem felvett                   0.0   
előkészítő_2022, tervezőgrafika - haladó felvett                       0.0   
                                         nem felvett                   0.0   
előkészítő_2022, tervezőgrafika - kezdő  felvett                       0.0   
                                         nem felvett                   0.0   
előkészítő_2022, textiltervezés          felvett                       0.0   
                                         nem felvett                   0.0   
előkészítő_2022, tárgyalkotás            felvett                       0.0   
                                         nem felvett                   0.0   
előkészítő_2022, építészet               nem felvett                   0.0   
intenzív 2022, animáció szakirány        nem felvett                   0.0   
intenzív 2022, formatervezés szakirány   felvett                       0.0   
intenzív 2022, fotográfia szakirány      felvett                       0.0   
intenzív 2022, rajz                      felvett                       0.0   
                                         nem felvett                   0.0   
intenzív 2022, tervezőgrafika szakirány  nem felvett                   0.0   
intenzív 2022, textil szakirány          felvett                       0.0   
                                         nem felvett                   0.0   
intenzív 2023, formatervezés             felvett                       0.0   
                                         nem felvett                   0.0   
intenzív 2023, média design              felvett                       0.0   
                                         nem felvett                   0.0   
intenzív 2023, rajz                      nem felvett                   0.0   
intenzív 2023, tervezőgrafika            felvett                       0.0   
                                         nem felvett                   0.0   
intenzív 2023, textiltervezés            felvett                       0.0   
                                         nem felvett                   0.0   
intenzív 2023, tárgyalkotás              felvett                       0.0   
                                         nem felvett                   0.0   

                                                              premomés  \
kurzus azonosító                         felvételi kategória             
nem járt                                 felvett                   0.0   
                 

In [149]:
def plot_barchart(prepcourse_df, coursevar_name='kurzus neve', save_dir='results/plots',
                  filename='felveteli_rata_plot.png'):

    data = prepcourse_df.reset_index().groupby(coursevar_name)['felvételi ráta'].max()
    data.sort_values(ascending=True, inplace=True)

    # Plot the horizontal bar graph
    ax = data.plot(kind='barh', align='center', color='C0')

    # Highlight zero-length bars in red
    for idx, value in enumerate(data):
        if value == 0:
            ax.get_children()[idx].set_color('red')
            ax.get_yticklabels()[idx].set_color('red')
        if value == 1:
            ax.get_children()[idx].set_color('green')
            ax.get_yticklabels()[idx].set_color('green')
        if data.index[idx] == 'nem járt':
            ax.get_children()[idx].set_color('yellow')
            ax.get_yticklabels()[idx].set_color('yellow')

    # Adjust x-axis limit for better visibility
    ax.set_xlim(0, 1)

    # Set smaller font size for y-axis labels
    ax.set_yticklabels(data.index, fontsize=8)

    # Make the y-axis label fancier by rotating it
    ax.set_ylabel(data.index.name, rotation=90, labelpad=20, fontsize=12)

    ax.set_title("Felvételi arány")

    plt.ioff()
    plt.figure(dpi=300)
    # Show the plot
    # Ensure the save directory exists, if not create it
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Save the plot as an HTML file
    file_path = os.path.join(save_dir, filename)

    plt.savefig(file_path, dpi=300, pad_inches=0.1)
    plt.close()
    #plt.show()

#
# ax = data.plot(kind='barh', align='center')
#
# # Adjust y-axis limit for better visibility
# ax.set_xlim(0, 1.05)
#
# # Set smaller font and rotation for x-axis labels
# ax.set_yticklabels(data.index, fontsize=8)
# ax.axvline(1.0, color='C0', linestyle='--')
# ax.set_ylabel(data.index.name, rotation=45, labelpad=20, fontsize=12)
#
# plt.show()

In [166]:
def interactive_scatter(df, coursevar_name='kurzus neve', save_dir='results/plots',
                        filename="interactive_scatter_with_line.html", analysis_type='felveteli'):
    rate = {'felveteli': 'felvételi ráta', 'jelentkezo': 'jelentkezési ráta'}[analysis_type]
    number_of_personnel = {'felveteli': 'felvételiző létszám', 'jelentkezo': 'teljes létszám'}[analysis_type]
    title = {'felveteli': 'Felvételi arányszámok', 'jelentkezo': 'Jelentkezési arányszámok'}[analysis_type]
    student_type = {'felveteli':'Felvett hallgató', 'jelentkezo':'Jelentkező hallgató'}[analysis_type]

    df = pd.merge(df.reset_index().groupby(coursevar_name)[rate].max(), df.reset_index().groupby(coursevar_name)[
        number_of_personnel].max(), left_index=True, right_index=True).reset_index()

    df[student_type] = df[rate] * df[number_of_personnel]

    # Ensure 'felvételi ráta' is in a proper numeric format
    df[rate] = df[rate].astype(str).apply(lambda x: x.replace(',', '.')).astype(float)

    # Create color conditions
    conditions = [
        (df[rate] == 1.0),
        (df[rate] == 0.0)

    ]

    # Color choices for each condition
    choices = ['green', 'red']

    # Create a new 'colors' column in the DataFrame
    df['colors'] = np.select(conditions, choices, default='blue')

    # Extract "nem járt" rate
    nem_jart_rate = df[df[coursevar_name] == 'nem járt'][rate].values[0]

    # Remove "nem járt" from the data for the scatter plot
    df = df[df[coursevar_name] != 'nem járt']

    # Create a scatter plot using plotly express
    fig = px.scatter(df,
                     x=number_of_personnel,
                     y=rate,
                     color='colors',
                     hover_name=coursevar_name,
                     hover_data=[student_type],
                     color_discrete_map='identity',
                     title=title,
                     labels={number_of_personnel: number_of_personnel.capitalize(),
                             rate: rate.capitalize()}
                     )

    if analysis_type == 'felveteli':
        # Adding a horizontal line for "nem járt"
        fig.add_trace(go.Scatter(x=[df[number_of_personnel].min(), df[number_of_personnel].max()],
                                 y=[nem_jart_rate, nem_jart_rate],
                                 mode='lines',
                                 line=dict(color='yellow', width=2),
                                 name='nem járt',
                                 hoverinfo='name+y'
                                 ))

    # Ensure the save directory exists, if not create it
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Save the plot as an HTML file
    file_path = os.path.join(save_dir, filename)
    fig.write_html(file_path)

    # Display the interactive plot
    fig.show()

# Example usage:
# Assuming `df` is your DataFrame containing the data
# interactive_scatter(df, 'results/plots')


In [162]:
def analize_acceptance_ratio(dataframe, save_dir='prezi/plots'):
    for analysis_type in ['felveteli', 'jelentkezo']:
        for coursevar_name in ['kurzus neve', 'kurzus azonosító']:
            print({'kurzus neve': 'Kurzus típusok szerint (harmonizáltan)',
                   'kurzus azonosító': 'Kurzusok szerint (nem harmonizáltan)'}[coursevar_name])
            course_df = gen_application_admission_dfs(dataframe, coursevar_name=coursevar_name)
            course_df.to_excel(f'results/felvételi arányok {coursevar_name} szerint.xlsx')
            plot_barchart(course_df, coursevar_name=coursevar_name,
                          filename=f'felveteli_rata_{coursevar_name.replace(" ", "_")}_barplot.png', save_dir=save_dir)
            interactive_scatter(course_df, coursevar_name=coursevar_name,
                                filename=f'{analysis_type}_rata_{coursevar_name.replace(" ", "_")}_interactive_plot.html',
                                save_dir=save_dir, analysis_type=analysis_type)

In [127]:
gen_application_admission_dfs(df)

nem premomés  \
kurzus azonosító                         felvételi kategória                 
nem járt                                 felvett                     150.0   
                                         nem felvett                1884.0   
7_alkalmas_őszi_2022, 7 alkalmas textil  felvett                       0.0   
                                         nem felvett                   0.0   
7_alkalmas_őszi_2022, 7 alklamas rajz    felvett                       0.0   
                                         nem felvett                   0.0   
előkészítő_2022, animáció                nem felvett                   0.0   
előkészítő_2022, designkultúra           felvett                       0.0   
előkészítő_2022, délelőtti rajz          nem felvett                   0.0   
előkészítő_2022, formatervezés           felvett                       0.0   
                                         nem felvett                   0.0   
előkészítő_2022, fotográfia              felvett                       0.0   
                                         nem felvett                   0.0   
előkészítő_2022, média design            felvett                       0.0   
                                         nem felvett                   0.0   
előkészítő_2022, rajzbérlet 10 alkalmas  felvett                       0.0   
                                         nem felvett                   0.0   
előkészítő_2022, rajzbérlet 19 alkalmas  felvett                       0.0   
                                         nem felvett                   0.0   
előkészítő_2022, tervezőgrafika - haladó felvett                       0.0   
                                         nem felvett                   0.0   
előkészítő_2022, tervezőgrafika - kezdő  felvett                       0.0   
                                         nem felvett                   0.0   
előkészítő_2022, textiltervezés          felvett                       0.0   
                                         nem felvett                   0.0   
előkészítő_2022, tárgyalkotás            felvett                       0.0   
                                         nem felvett                   0.0   
előkészítő_2022, építészet               nem felvett                   0.0   
intenzív 2022, animáció szakirány        nem felvett                   0.0   
intenzív 2022, formatervezés szakirány   felvett                       0.0   
intenzív 2022, fotográfia szakirány      felvett                       0.0   
intenzív 2022, rajz                      felvett                       0.0   
                                         nem felvett                   0.0   
intenzív 2022, tervezőgrafika szakirány  nem felvett                   0.0   
intenzív 2022, textil szakirány          felvett                       0.0   
                                         nem felvett                   0.0   
intenzív 2023, formatervezés             felvett                       0.0   
                                         nem felvett                   0.0   
intenzív 2023, média design              felvett                       0.0   
                                         nem felvett                   0.0   
intenzív 2023, rajz                      nem felvett                   0.0   
intenzív 2023, tervezőgrafika            felvett                       0.0   
                                         nem felvett                   0.0   
intenzív 2023, textiltervezés            felvett                       0.0   
                                         nem felvett                   0.0   
intenzív 2023, tárgyalkotás              felvett                       0.0   
                                         nem felvett                   0.0   

                                                              premomés  \
kurzus azonosító                         felvételi kategória             
nem járt                                 felvett                   0.0   
                 

In [167]:
analize_acceptance_ratio(df)

Kurzus típusok szerint (harmonizáltan)


Kurzusok szerint (nem harmonizáltan)


Kurzus típusok szerint (harmonizáltan)


Kurzusok szerint (nem harmonizáltan)


In [68]:
pd.DataFrame(
    df.groupby(['kurzus neve', 'előkészítő típusa', 'felvételi kategória'], dropna=False).size()).reset_index().pivot(
    index=['kurzus neve', 'felvételi kategória'], columns='előkészítő típusa', values=0)

előkészítő típusa                            nem premomés  premomés
kurzus neve             felvételi kategória                        
NaN                     felvett                     151.0       NaN
                        nem felvett                1883.0       NaN
SPARK Sprint            nem jelentkezett              NaN      20.0
animáció                nem felvett                   NaN      16.0
                        nem jelentkezett              NaN       5.0
designkultúra           felvett                       NaN       3.0
                        nem jelentkezett              NaN       1.0
formatervezés           felvett                       NaN      11.0
                        nem felvett                   NaN       7.0
                        nem jelentkezett              NaN       2.0
formatervezés szakirány felvett                       NaN       1.0
                        nem jelentkezett              NaN       3.0
fotográfia              felvett                       NaN       3.0
                        nem felvett                   NaN       3.0
                        nem jelentkezett              NaN       6.0
média design            felvett                       NaN       3.0
                        nem felvett                   NaN      13.0
                        nem jelentkezett              NaN       1.0
rajz                    felvett                       NaN       7.0
                        nem felvett                   NaN      37.0
                        nem jelentkezett              NaN      50.0
tervezőgrafika          felvett                       NaN      13.0
                        nem felvett                   NaN       9.0
                        nem jelentkezett              NaN      16.0
textiltervezés          felvett                       NaN       7.0
                        nem felvett                   NaN      15.0
                        nem jelentkezett              NaN      22.0
tárgyalkotás            felvett                       NaN       4.0
                        nem felvett                   NaN       3.0
                        nem jelentkezett              NaN       2.0
építészet               nem felvett                   NaN      10.0
                        nem jelentkezett              NaN       3.0

In [69]:
grouped = df.groupby(['előkészítő típusa', 'felvételi kategória'])

In [70]:
# Create a dictionary of DataFrames
dfs_by_merge = {}
for name, group in grouped:
    cleaned_group = group.dropna(axis=1, how='all')
    dfs_by_merge[name] = cleaned_group.reset_index(drop=True)


In [71]:
# Save to Excel, each DataFrame as a separate sheet
with pd.ExcelWriter('./results/jelentkezők szétválogatva.xlsx') as writer:
    for sheet_name, df_felvi_neptun in dfs_by_merge.items():
        df_felvi_neptun.to_excel(writer, sheet_name=', '.join(sheet_name), index=False)

In [85]:
df_subset_premome = df[df['előkészítő típusa'] == 'premomés'].copy()

In [89]:
df_subset_premome['hallgató képzése'] = df_subset_premome['hallgató képzése'].fillna(
    df_subset_premome['felvételi kategória'])

In [94]:
tab_harm = df_subset_premome.drop_duplicates(['id', 'kurzus neve'])
tab_full = df_subset_premome.drop_duplicates(['id', 'kurzus azonosító'])

In [99]:
# Create a crosstab to count transitions between states
pd.crosstab(tab_harm['hallgató képzése'], tab_harm['kurzus neve'], rownames=['egyetemi kurzus'],
            colnames=['előkészítő kurzus']).T.to_excel('./results/premome honnan hova harmonizált.xlsx')
pd.crosstab(tab_full['hallgató képzése'], tab_full['kurzus azonosító'], rownames=['egyetemi kurzus'],
            colnames=['előkészítő kurzus']).T.to_excel('./results/premome honnan hova teljes.xlsx')

In [105]:
df['felvételi kategória'].unique()

array(['nem premomés', 'premomés'], dtype=object)

## Generate Sankey

In [107]:
import itertools

In [151]:
def gen_states(dataframe, source_col: str = 'előkészítő típusa', target_col: str = 'felvételi kategória'):
    states = pd.DataFrame(
        list(itertools.product(dataframe[source_col].unique(), dataframe[target_col].unique())))
    states.columns = ['source', 'target']
    states['value'] = states.apply(lambda x: dataframe[
        dataframe[target_col].str.contains(f'^{x["target"]}') & dataframe[source_col].str.contains(
            f'^{x["source"]}')].drop_duplicates(['id']).shape[0], axis=1)
    return states


def visualize_sankey(df):
    source_factorized, source_labels = pd.factorize(df['source'])
    target_factorized, target_labels = pd.factorize(df['target'])

    # Update DataFrame with factorized source and target columns
    df['source_factorized'] = source_factorized
    df['target_factorized'] = target_factorized

    # Create lists for source, target, and values
    source_list = df['source_factorized'].tolist()
    target_list = df['target_factorized'].tolist()
    value_list = df['value'].tolist()

    # Combine source and target labels for unique labels
    all_labels = source_labels.tolist() + target_labels.tolist()

    # Adjust target indices to make them unique across source and target
    target_list_adjusted = [x + len(source_labels) for x in target_list]

    # Create the Sankey diagram
    fig = go.Figure(data=[go.Sankey(
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=all_labels,
        ),
        link=dict(
            source=source_list,
            target=target_list_adjusted,
            value=value_list,
        )
    )])

    #fig.show()
    return fig


In [143]:
states = gen_states(df)

In [160]:
fig = visualize_sankey(gen_states(df.drop_duplicates('id')))
fig.update_layout(title='Felvételi folyamatok alakulása')
fig.write_image("results/plots/sankey_diagram_teljes.png", scale=5)

In [156]:
df['hallgató képzése'] = df['hallgató képzése'].fillna(df['felvételi kategória'])

In [175]:
visualize_sankey(
    gen_states(df.drop_duplicates('id'), source_col='előkészítő típusa', target_col='hallgató képzése')).write_image(
    "results/plots/sankey_diagram_premome_hova.png", scale=5)

In [176]:
visualize_sankey(gen_states(df, source_col='kurzus neve', target_col='hallgató képzése')).write_image(
    "results/plots/sankey_diagram_premome_hurzus_hova.png", scale=5)